In [1]:
import yfinance as yf
import pandas as pd
import time

returns = pd.read_csv('data/returns.csv')
tickers = list(returns.columns)[1:]  

data = []

for ticker in tickers:
    try:
        info = yf.Ticker(ticker).info
        row = {
            'Ticker': ticker,
            'MarketCap': info.get('marketCap', 0),
            'Sector': info.get('sector', 'Unknown'),
            'Country': info.get('country', 'Unknown')
        }
        data.append(row)
        time.sleep(1.5)  
    except Exception as e:
        print(f"Error retrieving {ticker}: {e}")
        data.append({'Ticker': ticker, 'MarketCap': 0, 'Sector': 'Unknown', 'Country': 'Unknown'})


In [2]:
df = pd.DataFrame(data)

# Size classification
df['SmallCap'] = (df['MarketCap'] < 2e9).astype(int)
df['MidCap'] = ((df['MarketCap'] >= 2e9) & (df['MarketCap'] < 1e10)).astype(int)
df['LargeCap'] = (df['MarketCap'] >= 1e10).astype(int)



# Sector classification
df['Tech'] = df['Sector'].str.contains('Technology', case=False, na=False).astype(int)
df['Finance'] = df['Sector'].str.contains('Financial|Bank', case=False, na=False).astype(int)
df['Healthcare'] = df['Sector'].str.contains('Health', case=False, na=False).astype(int)
df['Consumer'] = df['Sector'].str.contains('Consumer', case=False, na=False).astype(int)
df['Energy'] = df['Sector'].str.contains('Energy|Oil|Gas', case=False, na=False).astype(int)
df['Industrial'] = df['Sector'].str.contains('Industrials', case=False, na=False).astype(int)
df['Utilities'] = df['Sector'].str.contains('Utilities', case=False, na=False).astype(int)

# Country classification
df['International'] = (df['Country'] != 'United States').astype(int)
df['Domestic'] = (df['Country'] == 'United States').astype(int)



In [3]:
binary_df = df[['Ticker', 'SmallCap', 'MidCap', 'LargeCap',
                'Tech', 'Finance', 'Healthcare', 'Consumer',
                'Energy', 'Industrial', 'Utilities',
                'International', 'Domestic']]

binary_df.to_csv("data/ticker_attributes.csv", index=False)


In [4]:

import pandas as pd
import numpy as np
import yfinance as yf
from amplpy import AMPL

def run_attribute_rebalanced_index(qs, ms):
    returns = pd.read_csv("data/returns.csv", index_col=0)
    sp100_returns = pd.read_csv("data/sp100returns.csv")
    combined = sp100_returns[['Portfolio_Return']].rename(columns={'Portfolio_Return': 'SP100'})

    binary_df = pd.read_csv("data/ticker_attributes.csv").set_index("Ticker")
    all_features = binary_df.columns.tolist()

    # Define fixed attribute targets 
    fixed_target_dict = {
        "SmallCap": .1,
        "MidCap": .1,
        "LargeCap": 1.0,
        "Tech": 0.6,
        "Finance": 0.8,
        "Healthcare": 0.07,
        "Consumer": 0.15,
        "Utilities": .2,
        "Energy": 0.02,
        "Industrial": 0.01,
        "Domestic": 1.0
    }

    for q in qs:
        for m in ms:
            print(f"\n>>> Processing q={q}, m={m}")
            split_point = int(0.7 * len(returns))
            out_sample = returns.iloc[split_point:].copy()
            n_rows = len(out_sample)
            period_length = n_rows // m
            periods = np.repeat(np.arange(1, m + 1), period_length)
            if n_rows % m != 0:
                periods = np.append(periods, [m] * (n_rows - len(periods)))
            out_sample['period'] = periods
            out_sample_tall = out_sample.reset_index().melt(
                id_vars=["Date", "period"], var_name="Ticker", value_name="Return"
            )

            is_windows = []
            for i in range(1, m + 1):
                start_date = out_sample[out_sample['period'] == i].index[0]
                is_window = returns.loc[:start_date].iloc[-split_point:]
                is_windows.append(is_window)

            all_portfolio_returns = []

            for i in range(m):
                window_returns = is_windows[i]
                correlations = window_returns.corr()
                tickers = list(correlations.columns)

                with open("data.txt", "w") as f:
                    f.write("set STOCKS := " + " ".join(tickers) + " ;\n\n")
                    f.write("param q := " + str(q) + " ;\n\n")
                    f.write("param r:\n    " + " ".join(tickers) + " :=\n")
                    for t1 in tickers:
                        row = " ".join(f"{correlations.loc[t1, t2]:.4f}" for t2 in tickers)
                        f.write(f"{t1} {row}\n")
                    f.write(";\n")

                ampl = AMPL()
                ampl.setOption("solver", "gurobi")
                ampl.read("max_corr.txt")
                ampl.readData("data.txt")
                ampl.solve()

                y = ampl.getVariable("y").getValues().to_pandas()
                selected = y[y["y.val"] == 1].index.tolist()

                if not selected:
                    print(f"(q={q}, m={m}) period {i+1}: No tickers selected — skipping.")
                    continue

                market_caps = {}
                for ticker in selected:
                    try:
                        info = yf.Ticker(ticker).info
                        market_caps[ticker] = info.get('marketCap', 0)
                    except:
                        market_caps[ticker] = 0
                total_cap = sum(market_caps.values())
                weights = {t: market_caps[t] / total_cap if total_cap > 0 else 0 for t in selected}

                available = [feat for feat in all_features if binary_df.loc[selected, feat].sum() > 0]
                if not available:
                    print(f"(q={q}, m={m}) period {i+1}: No usable features — skipping.")
                    continue

                
                target_dict = {feat: fixed_target_dict[feat] for feat in available if feat in fixed_target_dict}

                with open("attributes.dat", "w") as f:
                    f.write("set STOCKS := " + " ".join(selected) + " ;\n\n")
                    f.write("set FEATURES := " + " ".join(available) + " ;\n\n")

                    f.write("param x_orig :=\n")
                    for t in selected:
                        f.write(f"  {t} {weights[t]:.6f}\n")
                    f.write(";\n\n")

                    f.write("param a : " + " ".join(available) + " :=\n")
                    for t in selected:
                        row = " ".join(str(int(binary_df.loc[t, feat])) for feat in available)
                        f.write(f"{t} {row}\n")
                    f.write(";\n\n")

                    f.write("param f :=\n")
                    for feat in available:
                        f.write(f"  {feat} {target_dict[feat]:.6f}\n")
                    f.write(";\n")

                ampl = AMPL()
                ampl.setOption("solver", "gurobi")
                ampl.read("attributes.mod.txt")
                ampl.readData("attributes.dat")
                ampl.solve()

                x = ampl.getVariable("x").getValues().to_pandas()
                nonzero = x[x["x.val"] > 0].reset_index()
                nonzero = nonzero.rename(columns={nonzero.columns[0]: "Ticker", nonzero.columns[1]: "x.val"})
                nonzero["Weight"] = nonzero["x.val"]
                nonzero["period"] = i + 1
                all_portfolio_returns.append(nonzero)

                print(f"(q={q}, m={m}) period {i+1}: solution found with {len(nonzero)} non-zero weights")
                print(nonzero[["Ticker", "Weight"]].sort_values(by="Weight", ascending=False).to_string(index=False))

            if all_portfolio_returns:
                weights_df = pd.concat(all_portfolio_returns)
                portfolio = pd.merge(out_sample_tall, weights_df, on=['period', 'Ticker'], how='inner')
                portfolio['Weighted_Return'] = portfolio['Return'] * portfolio['Weight']
                portfolio_return = portfolio.groupby('Date')['Weighted_Return'].sum().reset_index()
                portfolio_return = portfolio_return.rename(columns={'Weighted_Return': 'Portfolio_Return'})
                label = f"(q={q}, m={m})"
                combined[label] = portfolio_return['Portfolio_Return']

    return combined


In [ ]:
combined_returns = run_attribute_rebalanced_index(qs=[5,25], ms=[2, 4])


>>> Processing q=5, m=2
Gurobi 12.0.1: optimal solution; objective 70.6261
894 simplex iterations
1 branching node
Gurobi 12.0.1: optimal solution; objective 0.41843
5 simplex iterations
(q=5, m=2) period 1: solution found with 5 non-zero weights
Ticker   Weight
  MSFT 0.600000
    KO 0.150000
   MET 0.145346
   JNJ 0.070000
   BLK 0.034654
Gurobi 12.0.1: optimal solution; objective 61.6403
1203 simplex iterations
1 branching node
Gurobi 12.0.1: optimal solution; objective 0.313368
5 simplex iterations
(q=5, m=2) period 2: solution found with 5 non-zero weights
Ticker   Weight
  MSFT 0.600000
   BLK 0.189268
   BAC 0.071643
   JNJ 0.070000
    KO 0.069089

>>> Processing q=5, m=4
Gurobi 12.0.1: optimal solution; objective 70.6261
894 simplex iterations
1 branching node
Gurobi 12.0.1: optimal solution; objective 0.41843
5 simplex iterations
(q=5, m=4) period 1: solution found with 5 non-zero weights
Ticker   Weight
  MSFT 0.600000
    KO 0.150000
   MET 0.145346
   JNJ 0.070000
   BLK 